In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('healthcare_reviews.csv')
df.head()

,Review_Text,Rating
0,I have mixed feelings about my experience.,4
1,The staff was caring and attentive. I couldn't...,5
2,I have mixed feelings about my experience.,5
3,I have mixed feelings about my experience.,5
4,The healthcare provider was excellent. I had a...,3


In [3]:
df.isnull().sum()

Review_Text    100
Rating           0
dtype: int64

In [4]:
df['Review_Text'].fillna('Not available', inplace=True)

In [5]:
df.isnull().sum()

Review_Text    0
Rating         0
dtype: int64

#### DataProcessing

#### created new column as Sentiment to consolidate the rating

##### 0-negative review, 1- neutral review, 2- positive review

In [6]:
df['Sentiment'] = df['Rating'].map({1:0,2:0,3:1,4:2,5:2})
print(df)

                                           Review_Text  Rating  Sentiment
0          I have mixed feelings about my experience.        4          2
1    The staff was caring and attentive. I couldn't...       5          2
2          I have mixed feelings about my experience.        5          2
3          I have mixed feelings about my experience.        5          2
4    The healthcare provider was excellent. I had a...       3          1
..                                                 ...     ...        ...
995  My experience was terrible. I would not recomm...       5          2
996  The service was disappointing. I won't be comi...       4          2
997    The service was okay, but nothing exceptional.        3          1
998        I have mixed feelings about my experience.        5          2
999  The staff was caring and attentive. I couldn't...       4          2

[1000 rows x 3 columns]


In [7]:
df.to_csv('healthcare_reviews_v1.csv', index=False, encoding='utf-8')

In [8]:
df.isnull().sum()

Review_Text    0
Rating         0
Sentiment      0
dtype: int64

#### checking the distribution of target column

In [9]:
df['Sentiment'].value_counts()

Sentiment
2    434
0    398
1    168
Name: count, dtype: int64

In [10]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix 

#### stopword package download

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Parthi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

#### Stemming

In [13]:
port_stem = PorterStemmer()

In [14]:
def stemming(review):
    stemmed_review = re.sub('[^a-zA-Z]',' ', review)
    stemmed_review = stemmed_review.lower()
    stemmed_review = stemmed_review.split()
    stemmed_review = [port_stem.stem(word) for word in stemmed_review if not word in stopwords.words('english')]
    stemmed_review = ' '.join(stemmed_review)

    return stemmed_review

In [15]:
df['stemmed_review'] = df['Review_Text'].apply(stemming)

In [16]:
df.head()

,Review_Text,Rating,Sentiment,stemmed_review
0,I have mixed feelings about my experience.,4,2,mix feel experi
1,The staff was caring and attentive. I couldn't...,5,2,staff care attent happier
2,I have mixed feelings about my experience.,5,2,mix feel experi
3,I have mixed feelings about my experience.,5,2,mix feel experi
4,The healthcare provider was excellent. I had a...,3,1,healthcar provid excel great experi


In [17]:
df.to_csv('stemmed_reviews.csv', index=False, encoding='utf-8')

In [18]:
print(df['Sentiment'].value_counts())

Sentiment
2    434
0    398
1    168
Name: count, dtype: int64


##### separate the column for model building

In [19]:
X= df['stemmed_review'].values
y = df['Sentiment'].values

#### Splitting the data into training and test set 

In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.20, stratify= y, random_state=2)

In [21]:
print(X.shape, X_train.shape,X_test.shape)

(1000,) (800,) (200,)


#### count vectorizer for converting text into numeric data

In [54]:
# from sklearn.feature_extraction.text import CountVectorizer

# count_vectorizer = CountVectorizer()
# X_train = count_vectorizer.fit_transform(X_train)
# X_test = count_vectorizer.transform(X_test)


#### TFIDF vectorizer for converting text data into numeric data

In [22]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

#### Training the Random Forest Classifier model

In [23]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)



RandomForestClassifier(random_state=42)

In [24]:
# Model Evaluation

predictions = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f"Test Accuracy: {accuracy}")

# Confusion Matrix

conf_matrix = confusion_matrix(y_test, predictions)
print(f"Confusion Matrix: {conf_matrix}")


Test Accuracy: 0.46
Confusion Matrix: [[34  0 45]
 [11  0 23]
 [29  0 58]]


#### Training the Logistic Regression model(classification model)

In [25]:
model = LogisticRegression(max_iter=1000)

In [26]:
model.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

#### Model evaluation

In [27]:
# Accuracy score
predictions = model.predict(X_train)
training_data_accuracy = accuracy_score(y_train,predictions)

In [28]:
print ('Accuracy score for training set:', training_data_accuracy)

Accuracy score for training set: 0.45


In [29]:
# Accuracy score
predictions = model.predict(X_test)
test_data_accuracy = accuracy_score(y_test,predictions)

In [30]:
print ('Accuracy score for test set:',test_data_accuracy)

Accuracy score for test set: 0.46


In [31]:
cm = confusion_matrix(y_test,predictions)
print(cm)

[[34  0 45]
 [11  0 23]
 [29  0 58]]


#### K_NN algorithms

In [32]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors= 5)
classifier.fit(X_train,y_train)

KNeighborsClassifier()

In [33]:
y_pred = classifier.predict(X_test)

In [34]:
# confusion matrix

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test,y_pred)
print(cm)

[[24  6 49]
 [ 8  1 25]
 [20  5 62]]


In [35]:
accuracy_score(y_test,y_pred)

0.435